# End to End Pure Streaming Data-Pipeline for Lease Table Using Spark Structured Streaming on Databricks

###### Description: In this notebook we read lease state rows from incoming csv files into a streamig dataframe, transform (clean, cast, rename) the data, add/update the latest state to a Databricks Delta table
###### Objective: (incoming csv files) --> "lease_streamingDF" --> "results_df" --> "lease_data"

In [3]:
import requests
import json
import optimus as op
import phonenumbers 
import re
import datetime
import time

from pyspark.sql.types import *
from pyspark.sql.functions import udf
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext, Row
from pyspark.sql.functions import unix_timestamp, from_unixtime
from pyspark.sql import functions as F
from pyspark.sql.window import Window as W
from pyspark.sql import DataFrame
from pyspark.sql.functions import lit
from pyspark.sql.functions import rank, col

In [4]:
# Schema for Lease 
lease_schema = StructType([
            StructField("Lease_id", IntegerType(), False),
            StructField("Start", StringType(), True),
            StructField("End", StringType(), False),
            StructField("Deposit", StringType(), True),
            StructField("Tenant_id", IntegerType(), True),
            StructField("Apartment_id", IntegerType(), True),
            StructField("event_time", TimestampType(), True)])

lease_udf_schema = StructType([
            StructField("Start", StringType(), True),
            StructField("End", StringType(), False),
            StructField("Deposit", StringType(), True),
            StructField("Tenant_id", IntegerType(), True),
            StructField("Apartment_id", IntegerType(), True),
            StructField("event_time", TimestampType(), True)])

###### Description: Get lease csv files as a streaming "lease_streamingDF" and process it on the fly and get transformed stream "lease_df"
###### Objective: (incoming csv files) --> "lease_streamingDF" --> "lease_df"

In [6]:
# Get lease Steaming DataFrame from csv files

# streaming starts here by reading the input files 
lease_Path = "/FileStore/apartment/lease/inprogress/"
lease_streamingDF = (
  spark
    .readStream
    .schema(lease_schema)
    .option("maxFilesPerTrigger", "1")
    .option("header", "true")
    .option("multiLine", "true")
    .csv(lease_Path)
)
# Clear invalid rows
lease_df = lease_streamingDF.select("*").where("Lease_id IS NOT NULL")
# Instantiation of DataTransformer class:
transformer = op.DataFrameTransformer(lease_df)
# Replace NA with 0's
transformer.replace_na(0.0, columns="*")
# Clear accents: clear_accents only from name column and not everywhere 
transformer.clear_accents(columns='*')
# Remove special characters:  From all Columns 
# transformer.remove_special_chars(columns=['lease_name', 'Address_line_1', 'City', 'Post_code', 'Region'])

##### This function parses the corresponding columns into a single column

In [8]:
def my_fun(Start, End, Deposit, Tenant_id, Apartment_id, event_time):
  return zip(Start, End, Deposit, Tenant_id, Apartment_id, event_time)

udf_Fun = udf(my_fun, ArrayType(lease_udf_schema))

In [9]:
intermediate_df = ( lease_df.withWatermark("event_time", "10 seconds")
            .groupBy("Lease_id")
            .agg(F.collect_list("Start").alias("Start"),
                 F.collect_list("End").alias("End"),
                 F.collect_list("Deposit").alias("Deposit"),
                 F.collect_list("Tenant_id").alias("Tenant_id"),
                 F.collect_list("Apartment_id").alias("Apartment_id"),
                 F.collect_list("event_time").alias("event_time"), 
                 F.max("event_time").alias("latest_event_time"))
            .select("Lease_id", 
                    F.explode(udf_Fun(F.column("Start"), 
                                      F.column("End"), 
                                      F.column("Deposit"), 
                                      F.column("Tenant_id"), 
                                      F.column("Apartment_id"),
                                      F.column("event_time")))
                    .alias("data"), "latest_event_time"))

##### Filter the data where event_time is latest

In [11]:
results_df = (intermediate_df
              .select("Lease_id", 
                      "data.Start", 
                      "data.End", 
                      "data.Deposit", 
                      "data.Tenant_id", 
                      "data.Apartment_id", 
                      "data.event_time", 
                      "latest_event_time")
              .where("data.event_time=latest_event_time")).orderBy("Lease_id")

##### Display final result
###### This result shows the latest state of all the unique lease_id

In [13]:
display(results_df)

##### Below cells are optional if external functionality or storage is needed

###### Write the stream to a Databricks Delta table for storage

In [16]:
streaming_query = (results_df.writeStream
 .format("delta")
 .outputMode("complete")
 .option("mergeSchema", "true")
 .option("checkpointLocation", "/delta/apartment/lease/_checkpoints/streaming-agg")
 .start("/delta/apartment/lease_data"))

#### Read the Delta Table as a Static or Streaming DataFrame
#### This dataframe wil always be Up-To-Date

In [18]:
lease_data = spark.read.format("delta").load("/delta/apartment/lease_data").orderBy("Lease_id")

In [19]:
display(lease_data)

### Do Some Live Streaming Graphs

In [21]:
lease_data_stream = spark.readStream.format("delta").load("/delta/apartment/lease_data")

In [22]:
display(lease_data_stream.groupBy("Apartment_id").count())